# XGBoost Versions
### Train bike rental dataset with two versions of XGBoost: 0.90-2 and 1.2-2

SageMaker SDK Documentation: https://sagemaker.readthedocs.io/en/latest/overview.html

In [1]:
import numpy as np
import pandas as pd

import boto3
import re

import sagemaker
from sagemaker import get_execution_role

## Upload Data to S3

In [2]:
# Specify your bucket name
bucket_name = 'chandra-ml-sagemaker'

training_folder = r'bikerental/training/'
validation_folder = r'bikerental/validation/'
test_folder = r'bikerental/test/'

s3_model_output_location = r's3://{0}/bikerental/model'.format(bucket_name)
s3_training_file_location = r's3://{0}/{1}'.format(bucket_name,training_folder)
s3_validation_file_location = r's3://{0}/{1}'.format(bucket_name,validation_folder)
s3_test_file_location = r's3://{0}/{1}'.format(bucket_name,test_folder)

In [3]:
print(s3_model_output_location)
print(s3_training_file_location)
print(s3_validation_file_location)
print(s3_test_file_location)

s3://chandra-ml-sagemaker/bikerental/model
s3://chandra-ml-sagemaker/bikerental/training/
s3://chandra-ml-sagemaker/bikerental/validation/
s3://chandra-ml-sagemaker/bikerental/test/


In [4]:
# Data Channels for the training algorithm
# content type can be libsvm or csv for XGBoost
training_input_config = sagemaker.session.TrainingInput(
    s3_data=s3_training_file_location,
    content_type='csv',
    s3_data_type='S3Prefix')

validation_input_config = sagemaker.session.TrainingInput(
    s3_data=s3_validation_file_location,
    content_type='csv',
    s3_data_type='S3Prefix'
)

data_channels = {'train': training_input_config, 'validation': validation_input_config}

In [5]:
# Write and Reading from S3 is just as easy
# files are referred as objects in S3.  
# file name is referred as key name in S3

# File stored in S3 is automatically replicated across 3 different availability zones 
# in the region where the bucket was created.

# http://boto3.readthedocs.io/en/latest/guide/s3.html
def write_to_s3(filename, bucket, key):
    with open(filename,'rb') as f: # Read in binary mode
        return boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(f)

In [6]:
write_to_s3('bike_train.csv', 
            bucket_name,
            training_folder + 'bike_train.csv')

write_to_s3('bike_validation.csv',
            bucket_name,
            validation_folder + 'bike_validation.csv')

write_to_s3('bike_test.csv',
            bucket_name,
            test_folder + 'bike_test.csv')

## Training Algorithm Docker Image
### SageMaker maintains a separate image for algorithm and region
https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html

In [7]:
# Use Spot Instance - Save up to 90% of training cost by using spot instances when compared to on-demand instances
# Reference: https://github.com/aws-samples/amazon-sagemaker-managed-spot-training/blob/main/xgboost_built_in_managed_spot_training_checkpointing/xgboost_built_in_managed_spot_training_checkpointing.ipynb

# if you are still on two-month free-tier you can use the on-demand instance by setting:
#   use_spot_instances = False

# We will use spot for training
use_spot_instances = True
max_run = 3600 # in seconds
max_wait = 7200 if use_spot_instances else None # in seconds

In [8]:
def fit_model(xgboostVersion):
    # Establish a session with AWS
    sess = sagemaker.Session()
    # This role contains the permissions needed to train, deploy models    
    role = get_execution_role()
    
    # job name has the version details - replace "." with "-"
    job_name = f"xgboost-bikerental-{xgboostVersion.replace('.','-')}"
    
    print (f"***Job name: {job_name}***")
    
    checkpoint_s3_uri = None

    if use_spot_instances:
        checkpoint_s3_uri = f's3://{bucket_name}/bikerental/checkpoints/{job_name}'

    print (f'Checkpoint uri: {checkpoint_s3_uri}')
            
    # Use the specified version of XGBoost
    container = sagemaker.image_uris.retrieve("xgboost",
                                              sess.boto_region_name,
                                              version=xgboostVersion)

    print (f'Using XGBoost Container {container}')
    
    # Configure the training job
    # for managed spot training, specify the use_spot_instances flag, max_run, max_wait and checkpoint_s3_uri
    estimator = sagemaker.estimator.Estimator(
        container,
        role,
        instance_count=1,
        instance_type='ml.m5.xlarge',
        output_path=s3_model_output_location,
        sagemaker_session=sess,
        base_job_name = job_name,
        use_spot_instances=use_spot_instances,
        max_run=max_run,
        max_wait=max_wait,
        checkpoint_s3_uri=checkpoint_s3_uri)

    estimator.set_hyperparameters(max_depth=5,
                              objective="reg:squarederror",
                              eta=0.1,
                              num_round=150)
    
    estimator.fit(data_channels)

    return estimator.latest_training_job.name

In [9]:
# XGBoost Algorithm Versions
xgBoostVersion = ["0.90-2", "1.2-2"]

In [10]:
jobName = []
for ver in xgBoostVersion:    
    jobName.append(fit_model(ver))

***Job name: xgboost-bikerental-0-90-2***
Checkpoint uri: s3://chandra-ml-sagemaker/bikerental/checkpoints/xgboost-bikerental-0-90-2
Using XGBoost Container 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3
2021-06-28 01:11:31 Starting - Starting the training job...
2021-06-28 01:11:33 Starting - Launching requested ML instancesProfilerReport-1624842691: InProgress
......
2021-06-28 01:12:49 Starting - Preparing the instances for training......
2021-06-28 01:13:49 Downloading - Downloading input data...
2021-06-28 01:14:33 Training - Training image download completed. Training in progress.
2021-06-28 01:14:33 Uploading - Uploading generated training model.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_cont


2021-06-28 01:14:50 Completed - Training job completed
Training seconds: 54
Billable seconds: 20
Managed Spot Training savings: 63.0%
***Job name: xgboost-bikerental-1-2-2***
Checkpoint uri: s3://chandra-ml-sagemaker/bikerental/checkpoints/xgboost-bikerental-1-2-2
Using XGBoost Container 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.2-2
2021-06-28 01:15:14 Starting - Starting the training job...
2021-06-28 01:15:16 Starting - Launching requested ML instancesProfilerReport-1624842914: InProgress
......
2021-06-28 01:16:40 Starting - Preparing the instances for training.........
2021-06-28 01:18:15 Downloading - Downloading input data
2021-06-28 01:18:15 Training - Downloading the training image...
2021-06-28 01:18:46 Uploading - Uploading generated training model
2021-06-28 01:18:46 Completed - Training job completed
[2021-06-28 01:18:34.767 ip-10-2-101-214.ec2.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-06-28:01:18:34:INFO] Imported framew

Training seconds: 56
Billable seconds: 20
Managed Spot Training savings: 64.3%


In [11]:
print(jobName)

['xgboost-bikerental-0-90-2-2021-06-28-01-11-31-651', 'xgboost-bikerental-1-2-2-2021-06-28-01-15-14-505']
